In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/MTP/DATASET')
print(os.getcwd())

/content
/content/drive/MyDrive/MTP/DATASET


In [ ]:
%pip install easyocr

     |████████████████████████████████| 70.8 MB 13 kB/s 
     |████████████████████████████████| 47.6 MB 1.7 MB/s 


In [ ]:
import cv2
import easyocr
import pickle
import os

def getTextCoordinates(image, reader):
    results = reader.readtext(image, paragraph = True)
    l = []
    
    for (bbox, text) in results:
        print(text, bbox)
        l.append((text, bbox))
    return l


def annotate_all():
    pathsrc = "DATASET/flowcharts_png"
    dictionary = {}
    fh = open('FullTextCoordinates', 'wb')
    reader = easyocr.Reader(['en'])
    
    for filename in os.listdir(pathsrc):
        path = os.path.join(pathsrc, filename)
        image = cv2.imread(path)
        textlist = getTextCoordinates(image, reader)
        dictionary[filename] = textlist
        print(len(dictionary))
        print("---------------------------------------------------------------------------------")
        
    print(dictionary)
    pickle.dump(dictionary, fh)
    fh.close()

 
    
annotate_all()

start fun1 [[246, 70], [492, 70], [492, 126], [246, 126]]
input: X [[273, 439], [467, 439], [467, 505], [273, 505]]
y = ((16 + x) - 20) [[147, 805], [591, 805], [591, 873], [147, 873]]
output: y [[257, 1177], [483, 1177], [483, 1241], [257, 1241]]
end function return [[134, 1542], [610, 1542], [610, 1598], [134, 1598]]
1
---------------------------------------------------------------------------------
start fun1o [[421, 67], [699, 67], [699, 127], [421, 127]]
input: radius [[400, 434], [720, 434], [720, 506], [400, 506]]
area = ((3.14 * radius) * radius) [[171, 796], [953, 796], [953, 872], [171, 872]]
output: area [[405, 1177], [715, 1177], [715, 1241], [405, 1241]]
end function return [[324, 1542], [798, 1542], [798, 1598], [324, 1598]]
2
---------------------------------------------------------------------------------
start fun1oo [[510, 166], [819, 166], [819, 226], [510, 226]]
input: N, k [[526, 635], [801, 635], [801, 708], [526, 708]]
sum = 0 [[559, 1110], [771, 1110], [771, 116

In [ ]:
import cv2
import pickle
import os

def getShapeCoordinates(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    shapes = {'Rectangle': [], 'Oval': [], 'Diamond': [], 'Parallelogram': []}
    i = 0
    for contour in contours:
        if i == 0:
            i = 1
            continue

        approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)
        area = cv2.contourArea(contour)

        if area > 1000:
            if len(approx) == 4:
                x, y = [], []
                d = {'++': 0, '+-': 0, '-+': 0, '--': 0}
                for l in contour:
                    pair = l[0]
                    x.append(pair[0])
                    y.append(pair[1])
                for j in range(len(x)):
                    if x[j] > x[j-1] and y[j] > y[j-1]:
                        d['++'] += 1
                    if x[j] > x[j-1] and y[j] < y[j-1]:
                        d['+-'] += 1
                    if x[j] < x[j-1] and y[j] > y[j-1]:
                        d['-+'] += 1
                    if x[j] < x[j-1] and y[j] < y[j-1]:
                        d['--'] += 1

                if (d['++'] == d['+-'] == d['-+'] == d['--'] == 0) or (d['++']==1 or d['+-']==1 or d['-+']==1 or d['--']==1):
                    shapes['Rectangle'].append(approx)

                elif (d['-+']==d['+-']==0 and abs(d['++']-d['--'])<5) or (abs(d['-+']-d['+-'])<5 and d['++']==d['--']==0):
                    shapes['Parallelogram'].append(approx)

                elif (abs(d['-+']-d['++'])<5 and abs(d['++']-d['+-'])<5 and abs(d['++']-d['--'])<5) :
                    shapes['Diamond'].append(approx)
                else:
                    pass

            elif len(approx) == 10:
                shapes['Oval'].append(approx)
                
    return shapes


def annotate_all():
    pathsrc = "DATASET/flowcharts_png"
    dictionary = {}
    fh = open('FullShapeCoordinates', 'wb')
    
    for filename in os.listdir(pathsrc):
        path = os.path.join(pathsrc, filename)
        image = cv2.imread(path)
        shapes = getShapeCoordinates(image)
        dictionary[filename] = shapes
        print(len(dictionary))
        print("---------------------------------------------------------------------------------")
        
    pickle.dump(dictionary, fh)
    fh.close()

 
    
annotate_all()


1
---------------------------------------------------------------------------------
2
---------------------------------------------------------------------------------
3
---------------------------------------------------------------------------------
4
---------------------------------------------------------------------------------
5
---------------------------------------------------------------------------------
6
---------------------------------------------------------------------------------
7
---------------------------------------------------------------------------------
8
---------------------------------------------------------------------------------
9
---------------------------------------------------------------------------------
10
---------------------------------------------------------------------------------
11
---------------------------------------------------------------------------------
12
-----------------------------------------------------------------------

In [ ]:
import pickle
fh = open('FullTextCoordinates', 'rb')
text_dict = pickle.load(fh)
fh.close()

fh = open('FullShapeCoordinates', 'rb')
shape_dict = pickle.load(fh)
fh.close()

#print(shape_dict)

def find_min_max(coordinate_list):
    min_x, min_y, max_x, max_y = coordinate_list[0][0], coordinate_list[0][1], coordinate_list[0][0], coordinate_list[0][1]
    for x, y in coordinate_list:
        if min_x > x:
            min_x = x
        if min_y > y:
            min_y = y
        if max_x < x:
            max_x = x
        if max_y < y:
            max_y = y
    return (min_x, min_y, max_x, max_y)

def is_within(text_coordinates, min_x, min_y, max_x, max_y):
    tl, tr, bl, br = text_coordinates[0], text_coordinates[1], text_coordinates[2], text_coordinates[3]
    flag = False
    if (min_x < tl[0] < max_x) and (min_x < tr[0] < max_x) and (min_x < bl[0] < max_x) and (min_x < br[0] < max_x) and (min_y < tl[1] < max_y) and (min_y < tr[1] < max_y) and (min_y < bl[1] < max_y) and (min_y < br[1] < max_y):
        flag = True
    return flag

def narray_to_list(narray):
    l = []
    for points in narray:
        l.append([points[0][0], points[0][1]])
    return l

def associate_shape_each(text, text_coordinates, shape_coordinates):
    SHAPE = None
    SHAPELIST = ['Rectangle', 'Diamond', 'Parallelogram', 'Oval']
    dcount = 0
    
    for shape in SHAPELIST:
        for narray in shape_coordinates[shape]:
            l = narray_to_list(narray)
            min_x, min_y, max_x, max_y = find_min_max(l)
            flag = is_within(text_coordinates, min_x, min_y, max_x, max_y)
            if flag:
                SHAPE = shape.upper()
        
    return SHAPE

def get_diamond_coordinates(text_coordinates, shape_coordinates):
    
    for narray in shape_coordinates['Diamond']:
        l = narray_to_list(narray)
        min_x, min_y, max_x, max_y = find_min_max(l)
        flag = is_within(text_coordinates, min_x, min_y, max_x, max_y)
        if flag:
            return l
        
def find_centroid(l):
    sum_x, sum_y = 0, 0
    for x, y in l:
        sum_x += x
        sum_y += y
    
    centroid = (sum_x//4, sum_y//4)
    return centroid

def find_distance(p1, p2):
    dis = (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2
    return dis

def associate_nearest_diamond(text_coord, diamond_coordinates):
    text_centroid = find_centroid(text_coord)
    min_dis = float('inf')
    nearest = None
    for diamond in diamond_coordinates:
        diamond_centroid = find_centroid(diamond_coordinates[diamond])
        dis = find_distance(text_centroid, diamond_centroid)
        if dis < min_dis:
            min_dis = dis
            nearest = diamond
    return nearest
        
def associate_shape(name):
    shape_coordinates = shape_dict[name]
    dcount = 0
    diamond_coordinates = {}
    text_shape_coord_list = []
    text_shape_list = []
    encoding = ''
    for text, text_coordinates in text_dict[name]:
        SHAPE = associate_shape_each(text, text_coordinates, shape_coordinates)
        if SHAPE == 'DIAMOND':
            dcount += 1
            SHAPE += str(dcount)
            diamond_coordinates[SHAPE] = get_diamond_coordinates(text_coordinates, shape_coordinates)
        text_shape_coord_list.append((text, SHAPE, text_coordinates))
        
        
    for text, shape, text_coord in text_shape_coord_list:
        if shape == None:
            SHAPE = associate_nearest_diamond(text_coord, diamond_coordinates)
            text_shape_list.append((text, SHAPE))
            if SHAPE == None:
                encoding += '{'+text+',None},'
            else:
                encoding += '{'+text+','+SHAPE+'},'
        else:
            text_shape_list.append((text, shape))
            encoding += '{'+text+','+shape+'},' 
    #print(text_shape_list)
    encoding = encoding[:-1]
    return (text_shape_list, encoding)
            

def annotate_all():
    pathsrc = "DATASET/flowcharts_png"
    dictionary_tuple = {}
    dictionary_string = {}

    for filename in os.listdir(pathsrc):
        encoding_tuple, encoding_string = associate_shape(filename)
        dictionary_tuple[filename] = encoding_tuple
        dictionary_string[filename] = encoding_string
        print(filename, dictionary_tuple[filename], dictionary_string[filename])
        print('------------------------------------------------')

    fh = open('flowchart_encodings_tuple', 'wb')
    pickle.dump(dictionary_tuple, fh)
    fh.close()
    fh = open('flowchart_encodings_string', 'wb')
    pickle.dump(dictionary_string, fh)
    fh.close()
    

annotate_all()
        

FileNotFoundError: ignored

In [ ]:
import pickle
fh = open('flowchart_encodings', 'rb')
encodings = pickle.load(fh)
fh.close()
print(encodings)

{'FLOW__PID_1.png': [('start fun1', 'OVAL'), ('input: X', 'PARALLELOGRAM'), ('y = ((16 + x) - 20)', 'RECTANGLE'), ('output: y', 'PARALLELOGRAM'), ('end function return', 'OVAL')], 'FLOW__PID_10.png': [('start fun1o', 'OVAL'), ('input: radius', 'PARALLELOGRAM'), ('area = ((3.14 * radius) * radius)', 'RECTANGLE'), ('output: area', 'PARALLELOGRAM'), ('end function return', 'OVAL')], 'FLOW__PID_100.png': [('start fun1oo', 'OVAL'), ('input: N, k', 'PARALLELOGRAM'), ('sum = 0', 'RECTANGLE'), ('=1', 'RECTANGLE'), ('no', 'DIAMOND1'), ('while (i <= N)', 'DIAMOND1'), ('output: sum', 'PARALLELOGRAM'), ('yes', 'DIAMOND1'), ('end function return', 'OVAL'), ('sum = (sum + i) if ((i % k) == 0)', 'RECTANGLE'), ('i = ( + 1)', 'RECTANGLE')], 'FLOW__PID_101.png': [('start fun101', 'OVAL'), ('input: a, d, N', 'PARALLELOGRAM'), ('sum = 0', 'RECTANGLE'), ('term = a', 'RECTANGLE'), ('no', 'DIAMOND1'), ('for in range(1, (N + 1))', 'DIAMOND1'), ('output: sum', 'PARALLELOGRAM'), ('yes', 'DIAMOND1'), ('end funct

In [ ]:
fh = open('DATASET/python_code/CODE_PID_1.py')
print(fh.readlines())
fh.close()

['def fun1(x):\n', '    y = 16 + x - 20\n', '    return y']


In [ ]:
import pickle

fh = open('encodings', 'rb')
encodings = pickle.load(fh)
fh.close()


In [ ]:
print(encodings['FLOW__PID_1.png'])

[('start fun1', 'OVAL'), ('input: X', 'PARALLELOGRAM'), ('y = ((16 + x) - 20)', 'RECTANGLE'), ('output: y', 'PARALLELOGRAM'), ('end function return', 'OVAL')]


In [ ]:
print(encodings['FLOW__PID_40.png'])

[('start fun4o', 'OVAL'), ('input: age', 'PARALLELOGRAM'), ('no', 'DIAMOND1'), ('if (age >= 18)', 'DIAMOND1'), ("output: 'Cannot vote'", 'PARALLELOGRAM'), ('yes', 'DIAMOND1'), ('end function return', 'OVAL'), ("output: 'Can vote", 'PARALLELOGRAM'), ('end function return', 'OVAL')]


In [ ]:
print(encodings['FLOW__PID_70.png'])

[('start fun7o', 'OVAL'), ('input: num', 'PARALLELOGRAM'), ('sum = 0', 'RECTANGLE'), ('no', 'DIAMOND1'), ('while (num != 0)', 'DIAMOND1'), ('output: sum', 'PARALLELOGRAM'), ('yes', 'DIAMOND1'), ('end function return', 'OVAL'), ('d = (num % 10)', 'RECTANGLE'), ('sum = (sum + d)', 'RECTANGLE'), ('num = (num // 10)', 'RECTANGLE')]
